In [2]:
pip install selenium

  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 4.1 MB/s eta 0:00:004.1 MB/s eta 0:00:01
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bs4

  Using cached beautifulsoup4-4.13.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.3-py3-none-any.whl (186 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver_manager

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
#import thu vien
import os 
import re
import ast
import argparse
import logging
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime

In [6]:
logging.basicConfig(
    filename='log/app.log', 
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.info("------Log file --------")


def chrome_webdriver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Chạy Chrome không hiển thị giao diện
    options.add_argument("--disable-gpu")  # Tắt GPU tăng hiệu suất
    options.add_argument("--no-sandbox")  # Tránh lỗi sandbox trong môi trường Linux

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [7]:
def craw_technique(browser, url):
    try:
        browser.get(url)
        sleep(5)
        list_thumb = WebDriverWait(browser, 5).until(
            EC.presence_of_element_located((By.ID, "Listthumb"))
        )
        # Tìm phần tử con và click vào đó
        technique_info = list_thumb.find_element(By.XPATH, '//*[@id="Listthumb"]/a/p')
        browser.execute_script("arguments[0].click();", technique_info)
        spec_items =  WebDriverWait(browser, 5).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "specs-item"))
        )
        res = []
        for spec_content in spec_items:
            all_li = spec_content.find_elements(By.CSS_SELECTOR, "ul.specs-content li")
            for li in all_li:
                span = li.find_element(By.TAG_NAME, "span").text
                strong = li.find_element(By.TAG_NAME, "strong").text
                res.append({strong : span})
        info_technique = {
            'url': url,
            'Technique': res
        }
        return info_technique
    except Exception as e:
        logging.warning(f'Lỗi: {e}')
        return []

In [9]:
import pandas as pd
data = pd.read_csv('data/laptop.csv')
data['url'].head()

0    https://hoanghamobile.com/laptop/mac-studio-ch...
1    https://hoanghamobile.com/laptop/macbook-air-m...
2    https://hoanghamobile.com/laptop/macbook-air-m...
3    https://hoanghamobile.com/laptop/macbook-air-m...
4    https://hoanghamobile.com/laptop/macbook-air-m...
Name: url, dtype: object

In [10]:
browser = chrome_webdriver()

if browser:
    data['Technique'] = data['url'].apply(lambda url: craw_technique(browser, url))
    browser.close()
else:
    print("Không thể khởi chạy trình duyệt")

In [12]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
data.to_csv('data/laptop(new).csv', index= False)
data.to_excel('data/laptop(new).xlsx', index = False)